In [26]:
import duckdb
import pandas as pd
from nba_api.stats.endpoints import commonallplayers, leaguedashplayerstats

In [69]:
# Pull players into a dataframe

players = commonallplayers.CommonAllPlayers(is_only_current_season=0)

df_players = players.get_data_frames()[0]

df_players.head(10)

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,alaa_abdelnaby,0,,,,,None,Y,00
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,zaid_abdul-aziz,0,,,,,None,Y,00
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,kareem_abdul-jabbar,0,,,,,None,Y,00
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,mahmoud_abdul-rauf,0,,,,,None,Y,00
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,tariq_abdul-wahad,0,,,,,None,Y,00
5,949,"Abdur-Rahim, Shareef",Shareef Abdur-Rahim,0,1996,2007,shareef_abdur-rahim,shareef_abdur-rahim,0,,,,,None,Y,00
6,76005,"Abernethy, Tom",Tom Abernethy,0,1976,1980,HISTADD_tom_abernethy,tom_abernethy,0,,,,,None,Y,00
7,76006,"Able, Forest",Forest Able,0,1956,1956,HISTADD_frosty_able,forest_able,0,,,,,None,Y,00
8,76007,"Abramovic, John",John Abramovic,0,1946,1947,HISTADD_brooms_abramovic,john_abramovic,0,,,,,None,Y,00
9,203518,"Abrines, Alex",Alex Abrines,0,2016,2018,alex_abrines,alex_abrines,0,,,,,None,Y,00


In [68]:
# Pull player stats for 2024-2025 season

player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2024-25', per_mode_detailed='PerGame')

df_player_stats = player_stats.get_data_frames()[0]

df_player_stats.head(10)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
0,1630639,A.J. Lawson,A.J.,1610612761,TOR,24.0,26,14,12,0.538,...,330,201,247,207,219,300,292,159,44,281
1,1631260,AJ Green,AJ,1610612749,MIL,25.0,73,44,29,0.603,...,467,520,136,394,278,83,342,281,44,318
2,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,29,8,21,0.276,...,449,120,239,321,273,522,332,281,44,328
3,203932,Aaron Gordon,Aaron,1610612743,DEN,29.0,51,33,18,0.647,...,290,84,277,89,98,26,127,134,44,124
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,28.0,62,39,23,0.629,...,385,415,413,350,363,142,422,281,44,403
5,1630174,Aaron Nesmith,Aaron,1610612754,IND,25.0,45,29,16,0.644,...,231,182,52,169,147,53,193,281,44,180
6,1630598,Aaron Wiggins,Aaron,1610612760,OKC,26.0,76,62,14,0.816,...,327,170,349,326,146,14,187,159,44,177
7,1641745,Adam Flagler,Adam,1610612760,OKC,25.0,37,34,3,0.919,...,478,514,521,555,514,222,527,281,44,523
8,1641766,Adama Sanogo,Adama,1610612741,CHI,23.0,4,2,2,0.500,...,519,348,540,521,503,367,526,281,44,524
9,1641737,Adem Bona,Adem,1610612755,PHI,22.0,58,12,46,0.207,...,30,373,134,192,355,404,312,134,44,334


## Load raw data into duckdb

##  Bronze Layer – Raw Ingestion

The Bronze layer represents the raw ingested data, pulled directly from the nba_api.
At this stage, data is untransformed and is written directly into DuckDB from pandas DataFrames.

These are stored under the raw schema and form the foundation for all transformations.

In [72]:
sql_raw_players_table = """
create or replace table raw.players as
select * from df_players
"""

sql_raw_player_stats_table = """
create or replace table raw.player_stats as 
select * from df_player_stats
"""

with duckdb.connect('../duckdb/nba.duckdb') as con:
    con.execute("create schema if not exists raw")
    con.execute(sql_raw_players_table)
    con.execute(sql_raw_player_stats_table)


In [73]:
with duckdb.connect('../duckdb/nba.duckdb') as con:
    display(con.sql("select * from raw.player_stats limit 5").df())

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
0,1630639,A.J. Lawson,A.J.,1610612761,TOR,24.0,26,14,12,0.538,...,330,201,247,207,219,300,292,159,44,281
1,1631260,AJ Green,AJ,1610612749,MIL,25.0,73,44,29,0.603,...,467,520,136,394,278,83,342,281,44,318
2,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,29,8,21,0.276,...,449,120,239,321,273,522,332,281,44,328
3,203932,Aaron Gordon,Aaron,1610612743,DEN,29.0,51,33,18,0.647,...,290,84,277,89,98,26,127,134,44,124
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,28.0,62,39,23,0.629,...,385,415,413,350,363,142,422,281,44,403


## Silver Layer – Cleaned & Normalised

The Silver layer contains cleaned, typed, and normalised tables.
This includes renaming columns, removing unnecessary fields, converting types (e.g., string to int), and preparing the data for analysis.

- player_stats_cleaned includes renamed columns like points_per_game, field_goal_percentage

- players_cleaned includes player metadata with fields like full_name, career_start_year, and is_active

In [74]:
with duckdb.connect('../duckdb/nba.duckdb') as con:
    display(con.sql("select player_name, points_per_game, field_goal_percentage from player_stats_cleaned limit 5").df())


,player_name,points_per_game,field_goal_percentage
0,A.J. Lawson,9.1,0.421
1,AJ Green,7.4,0.429
2,AJ Johnson,7.6,0.385
3,Aaron Gordon,14.7,0.531
4,Aaron Holiday,5.5,0.437


## Gold Layer – Enriched & Analytics-Ready

The Gold layer contains analytical models ready for direct consumption in reports, dashboards, or notebooks.
This includes aggregations, rankings, and joins of Silver-layer data.
The MVP model top_scorers_by_season ranks players by points per game for the 2024–2025 season.

In [75]:
with duckdb.connect('../duckdb/nba.duckdb') as con:
    display(con.sql("""
        select * from top_scorers_by_season
    """).to_df())

,player_id,full_name,team_name,points_per_game,games_played,season,player_rank
0,1628983,Shai Gilgeous-Alexander,Thunder,32.7,76,2024-2025,1
1,203507,Giannis Antetokounmpo,Bucks,30.4,67,2024-2025,2
2,203999,Nikola Jokić,Nuggets,29.6,70,2024-2025,3
3,1629029,Luka Dončić,Lakers,28.2,50,2024-2025,4
4,1630162,Anthony Edwards,Timberwolves,27.6,79,2024-2025,5
5,1628369,Jayson Tatum,Celtics,26.8,72,2024-2025,6
6,201142,Kevin Durant,Suns,26.6,62,2024-2025,7
7,1630178,Tyrese Maxey,76ers,26.3,52,2024-2025,8
8,1630595,Cade Cunningham,Pistons,26.1,70,2024-2025,9
9,1628973,Jalen Brunson,Knicks,26.0,65,2024-2025,10


### MVP Summary

This notebook builds an end-to-end dbt analytics pipeline using the NBA API, DuckDB, and dbt:

- Ingests player metadata and season stats for 2024–25
- Normalises and cleans both datasets in the Silver layer
- Joins and ranks players in the Gold layer by points per game
- Final table: `top_scorers_by_season`
